In [ ]:
from pathlib import Path
current_directory = Path.cwd()
dataset_folder = current_directory.parent.joinpath("end-to-end-rag", 'datasets', 'halifax_intermediaries')

In [ ]:
file_path = dataset_folder.joinpath("mortgage-guide.pdf")

In [ ]:
from openparse import DocumentParser

In [ ]:
document_parser = DocumentParser(table_args={"parsing_algorithm": "pdfplumber",})

In [ ]:
import openparse

In [ ]:
pdf_doc = openparse.Pdf(file=file_path)

In [ ]:
document = document_parser.parse(file_path, ocr=False, )

So I can use openparse with the default parser to detect the pages and then use py

Dealing with the duplicates table nodes

In [ ]:
import re

In [ ]:
def chunk_text(text: str) -> list[dict]:
    """
    Chunks a given string into groups of paragraphs with a corresponding header.

    The function assumes:
    - Paragraphs are delimited by '<br><br>' or '.\n'.
    - Headers are text enclosed in '** **' at the beginning of a paragraph.
    - Multiple paragraphs can belong to a single header.

    Args:
        text: The raw string content parsed from a PDF.

    Returns:
        A list of dictionaries, where each dictionary represents a chunk and
        contains 'header' and 'content' keys. The 'content' key holds a list of
        strings, with each string being a single paragraph.
    """
    chunks = []
    # Split the text by either '<br><br>' or '.\n' using a regex
    paragraphs = re.split(r'(?:<br><br>|\.\s*\n)', text.strip())

    current_header = ""
    current_content = []

    for p in paragraphs:
        p_stripped = p.strip()

        # Regex to capture the entire header block and the rest of the text
        header_match = re.match(
            r'^((\*\*(.*?)\*\*[\s\n]*)+)(.*)', p_stripped, re.DOTALL)

        if header_match:
            # If a new header is found, and we have content from a previous chunk,
            # store the previous chunk and reset.
            if current_content:
                chunks.append({
                    'header': current_header,
                    'content': current_content
                })

            # Extract the entire header block
            header_block = header_match.group(1).strip()
            # Extract the content that follows the header block
            content_part = header_match.group(4).strip()

            # Find all individual headers within the block and join them
            header_parts = re.findall(r'\*\*(.*?)\*\*', header_block)
            current_header = " ".join(header_parts)

            current_content = [content_part] if content_part else []

        else:
            # If no header, this paragraph belongs to the current header
            if p_stripped:  # Avoid appending empty strings
                current_content.append(p_stripped)

    # Add the last chunk after the loop finishes
    if current_content:
        chunks.append({
            'header': current_header,
            'content': current_content
        })

    return chunks

In [ ]:
def chunk_markdown_table(table_string):
    """
    Chunks a markdown table string by row, handling empty header columns.

    Args:
        table_string: A string containing a markdown table.

    Returns:
        A list of strings, where each string represents a chunked row.
    """
    lines = table_string.strip().split('|\n|')
    

    # Extract headers from the first line, handling multiple spaces and empty columns
    header_line = lines[0]
    # Just remove the first which can be empty sometimes
    headers = [h for h in header_line.split('|') if len(h) > 0]


    data_rows = lines[2:] # remove the header and separator lines

    chunked_rows = []
    for row in data_rows:
        # Split row by '|' and clean up empty strings
        values = [v.strip() for v in row.split('|') if v.strip()]

        if len(headers) == len(values):
            chunk = '; '.join(
                [f"{headers[i]}: {values[i]}" for i in range(len(headers))])
            chunked_rows.append(chunk)

    return chunked_rows

In [ ]:
sample_text = document.nodes[4]

In [ ]:
sample_text.text

In [ ]:
chunks = chunk_text(sample_text.text)

In [ ]:
print(chunks[0].get("content")[0].strip().split('\n')[0])

In [ ]:
all_chunks = []
for node in document.nodes:
    chunks = chunk_text(node.text)
    if node.variant in [{"table"}, {"text", "table"}]:
        for chunk in chunks:
            header = chunk['header']
            for table in chunk.get("content", ""):
                table_chunks = chunk_markdown_table(table)
                for table_chunk in table_chunks:
                    all_chunks.append({
                        'header': header,
                        'content': table_chunk
                    })
    else:
        all_chunks.extend(chunks)

Done with the text pdf processign on the page, need now to work out the code for the chunking.

In [ ]:
len(all_chunks)

In [ ]:
import random

In [ ]:


random.choice(all_chunks)